In [ ]:
# 用money/刷卡次数/平均单次消费为聚类字段
# data1['day']=pd.DatetimeIndex(data1['Date']).day
q1=[]
q2=[]
q3=[]
for i in data1['CardNo'].value_counts().index:
    major=data1[data1['CardNo']==i]
    q1.append(i)#卡号
    q2.append(sum(major['Money']))#卡号对应的money
    q3.append(len(major['Money'])-sum(major.duplicated(['day','hour'])))#刷卡频次统计：刷卡时间相近的算一次消费
# print(q1)
# print(q2)
s1=pd.Series(q1,index=range(1,len(q1)+1),name='学号')
s2=pd.Series(q2,index=range(1,len(q2)+1),name='金额')
s3=pd.Series(q3,index=range(1,len(q3)+1),name='刷卡次数')
df=pd.DataFrame({s1.name:s1,s2.name:s2,s3.name:s3})
df.describe()
#对要聚类的字段进行数据处理：消除一些刷卡次数少的人以及一些异常值
a = df["刷卡次数"].quantile(0.75)
b = df["刷卡次数"].quantile(0.25)
c = df["金额"].quantile(0.75)
d = df["金额"].quantile(0.25)
demoney =df["刷卡次数"]
jine=df["金额"]
de=df[(jine>=(c-d)*1.5+c)|(demoney<=b)].index
df=df.drop(de)
df["平均单次消费"]=df["金额"]/df["刷卡次数"]
def ZscoreNormalization(x):#标准化
    """Z-score normaliaztion"""
    x = (x - np.min(x)) / (np.max(x)-np.min(x))
    return x
df['金额']=ZscoreNormalization(df['金额'])
df['刷卡次数']=ZscoreNormalization(df['刷卡次数'])
df['平均单次消费']=ZscoreNormalization(df['平均单次消费'])

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
import matplotlib.pyplot as plt

d=df[['金额','刷卡次数','平均单次消费']]
mod = KMeans(n_clusters=4, max_iter = 1000)#聚成3类数据,并发数为4，最大循环次数为500
mod.fit_predict(d)#y_pred表示聚类的结果

r1 = pd.Series(mod.labels_).value_counts()
r2 = pd.DataFrame(mod.cluster_centers_)
r = pd.concat([r2, r1], axis = 1)
r.columns = list(d.columns) + [u'类别数目']
print(r)
#给每一条数据标注上被分为哪一类
r = pd.concat([d, pd.Series(mod.labels_, index = d.index)], axis = 1)
r.columns = list(d.columns) + [u'聚类类别']
print(r)